# Transcrição de áudio com o Azure Speech Services

## Fluxo básico

1. Os arquivos de áudio a serem transcritos devem estar em uma URL de acesso público ou em uma storage account (neste caso, deve ser provida uma URL com SAS - shared access signature - para que haja permissão de acesso);
2. Acionar o endpoint de criar transcrição para iniciar o job;
3. Consultar o resultado da transcrição, acionando o endpoint em intervalos periódicos, até a conclusão do job;
4. Consultar a lista de arquivos gerados para a construção;
5. Consultar o arquivo de transcrição.

In [1]:
import os
import requests
import json
import time
import urllib3
from urllib.parse import urlparse
from configparser import ConfigParser, ExtendedInterpolation

# Configura o certificado Petrobras para a lib requests
PTB_CERT_PATH = os.path.join(os.path.abspath(''), '../petrobras-ca-root.pem')
os.environ['REQUESTS_CA_BUNDLE'] = PTB_CERT_PATH

session = requests.Session()

# Obtém as credenciais do arquivo de config
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../config.ini', 'UTF-8')

# URL base da API no Hub de Modelos (APIM)
BASE_URL = 'https://apid.petrobras.com.br/ia/voz/v1/azure-speech-services'

# Headers enviado nas requisições
default_headers = {
    "Content-Type": "application/json",
    "apikey": config['HUB']['apikey_modelos_voz'],
}


# Método utilitário para fazer uma requisição GET e retornar o body em JSON
# Em caso de status diferente de sucesso (200), lança uma exceção
def get_json_checking_error(url, use_apim = True, verify_certificate=True):
    if use_apim:
        parsed_url = urlparse(url)
        azure_path_start = parsed_url.path.index('/speechtotext')
        url = f'{BASE_URL}{parsed_url.path[azure_path_start:]}'
    print(f'GET {url}')
    response = session.get(url, headers=default_headers, verify=verify_certificate)
    if response.status_code != 200:
        raise RuntimeError(f'Erro no response ({response.status_code}): {response.json()}')
    return response.json()


def consultar_transcricao(url, wait_time_seconds=3, max_retries=10):

    for i in range(max_retries):

        body = get_json_checking_error(url)
        job_status = body.get('status', '')
        if job_status == 'Succeeded':

            # obteve sucesso, então consulta os arquivos gerados pelo job
            url_arquivos = body['links']['files']
            body = get_json_checking_error(url_arquivos)

            # seleciona o arquivo que contém a transcrição
            for file in body['values']:
                if file['kind'] == 'Transcription':
                    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
                    return get_json_checking_error(file['links']['contentUrl'], False, False)

        elif job_status == 'Running':
            print(f'Job ainda em execução... esperando mais {wait_time_seconds}s')
            time.sleep(wait_time_seconds)
        else:
            raise RuntimeError(f'Erro no job: {body}')


def criar_transcricao(audio_url):

    # Dados de entrada do modelo
    input = {
        'contentUrls': [audio_url],
        'locale': 'pt-BR',
        'displayName': 'Exemplo 1',
        'properties': {
            'wordLevelTimestampsEnabled': True,
            'languageIdentification': {
                'candidateLocales': [
                    'pt-BR',
                    'en-US'
                ]
            }
        }
    }

    print(f'Criando transcrição: {json.dumps(input)}')

    response = session.post(f'{BASE_URL}/speechtotext/v3.1/transcriptions', headers=default_headers, json=input)

    if response.status_code != 201:
        raise RuntimeError(f'Erro na requisição ({response.status_code}): {json.dumps(response.json(), indent=4)}')
    else:
        body = response.json()
        return body['self']



In [2]:

# O arquivo de áudio deve estar em uma URL pública ou em um storage account
# da Azure (neste caso, é preciso usar uma URL com SAS - shared access signature)
SOURCE_AUDIO_URL = 'https://crbn.us/whatstheweatherlike.wav'

url_transcricao = criar_transcricao(SOURCE_AUDIO_URL)
transcricao = consultar_transcricao(url_transcricao)

print(f'Resultado: {json.dumps(transcricao, indent=4)}')

Criando transcrição: {"contentUrls": ["https://crbn.us/whatstheweatherlike.wav"], "locale": "pt-BR", "displayName": "Exemplo 1", "properties": {"wordLevelTimestampsEnabled": true, "languageIdentification": {"candidateLocales": ["pt-BR", "en-US"]}}}
GET https://apid.petrobras.com.br/ia/voz/v1/azure-speech-services/speechtotext/v3.1/transcriptions/3dbcf363-4b4f-43ae-a68e-77c26a7e62b1
Job ainda em execução... esperando mais 3s
GET https://apid.petrobras.com.br/ia/voz/v1/azure-speech-services/speechtotext/v3.1/transcriptions/3dbcf363-4b4f-43ae-a68e-77c26a7e62b1
Job ainda em execução... esperando mais 3s
GET https://apid.petrobras.com.br/ia/voz/v1/azure-speech-services/speechtotext/v3.1/transcriptions/3dbcf363-4b4f-43ae-a68e-77c26a7e62b1
GET https://apid.petrobras.com.br/ia/voz/v1/azure-speech-services/speechtotext/v3.1/transcriptions/3dbcf363-4b4f-43ae-a68e-77c26a7e62b1/files
GET https://spsvcprodbrs.blob.core.windows.net/bestor-cris/TranscriptionData/3dbcf363-4b4f-43ae-a68e-77c26a7e62b1_0